In [ ]:
## if it's your first run please install the following packages
#!pip install opencv-python
#!pip install matplotlib
#!pip install tensorflow

In [ ]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

## functions

In [ ]:
def check_image_svm(face_image,model):
    img_size=64
    final_image = cv2.resize(face_image, (img_size,img_size))
    final_image = np.array(final_image, np.float32).reshape(-1,img_size, img_size, 3)
    final_image = final_image/255.0
    Predictions = model.predict(final_image)
    if(Predictions<0):## negative emotion
        return 0
    else:
        return 1 ##positive emotion 
    
def check_image_cnn(face_image,model):## checks face image with cnn model and returns the emotion num
    img_size=48
    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    final_image = cv2.resize(face_image, (img_size,img_size))
    final_image = np.array(final_image, np.float32).reshape(-1,img_size, img_size, 1)
    final_image = final_image/255.0
    Predictions = model.predict(final_image)
    result = np.argmax(Predictions)
    return result
    
def get_cnn_model(result_svm,cnn_model_positive,cnn_model_negative):
    if(result_svm==0):
        return cnn_model_negative
    else:
        return cnn_model_positive
    
def get_color_and_classes(result_svm,classes_Positive,classes_Negative,colors):
    if(result_svm==0):
        return (colors[0],classes_Negative)
    else:
        return (colors[1],classes_Positive)
    

## main

In [ ]:
classes_Positive = ["happy", "neutral", "surprise"]  # list of positive emotions
classes_Negative = ["angry",  "fear", "sad"]  # list of negative emotions
cnn_model_positive = tf.keras.models.load_model('best_positive_model.h5')
cnn_model_negative = tf.keras.models.load_model('best_negative_model.h5')
svm_model = tf.keras.models.load_model('model_pos_neg.h5')
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
line_thickness=10
font = cv2.FONT_HERSHEY_PLAIN
colors=((0, 0, 255),(0, 255, 0),(255, 0, 0))## red,green,blue
text_placement=(int(line_thickness+5),int(line_thickness+30))
emotion_num = None
video_capture = cv2.VideoCapture(0)
width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH )
height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT )

while True:
    if not video_capture.isOpened():
        print('Unable to load camera.')
        sleep(5)
        pass

    # Capture frame-by-frame
    ret, frame = video_capture.read()
    cropped_frame = None
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(80, 80))
    if ret:
        cv2.rectangle(frame, (0,0), (int(width), int(height)), colors[2],line_thickness)
        for (x, y, w, h) in faces:
            cropped_frame = frame[y:y + h,x:x + w ]## cut face image for svm and cnn
        if cropped_frame is not None: #check if cropped_frame is defined
            if cropped_frame.any():# check if array is not empty
                # Display the result emotion 
                result_svm=check_image_svm(cropped_frame,svm_model)
                model=get_cnn_model(result_svm,cnn_model_positive,cnn_model_negative)
                emotion_num = check_image_cnn(cropped_frame,model)
                color,classes = get_color_and_classes(result_svm,classes_Positive,classes_Negative,colors)
                status = classes[emotion_num]
                cv2.putText(frame, status,text_placement,font, 3,color,2,cv2.LINE_4)
                cv2.rectangle(frame, (0,0), (int(width), int(height)), color,line_thickness)
                ##select_emotion_to_print_on_screen(frame,0,emotion_num,classes_Positive)## need to change veriables
              
        cv2.imshow('Emotion detection project', frame)## display camera image

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the captureq
video_capture.release()
cv2.destroyAllWindows()